# Zillow Project  Workbook

-------------------------------------------------



## Import Menagerie

In [ ]:
# Foundation
import pandas as pd
import numpy as np
from scipy import math

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns


# Metric Tools
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing


# Custom
import wrangle as wrg
import evaluate as eva
import explore as exp